# Project 2 - Manatee Call Detection
*Ian Malone

## Project Description

The purpose of this project is to design and evaluate a machine learning detection approach to distinguish the manatee calls from the background.

The data sets as follows: 
(1) a file (train_signal.wav) with 10 different manatee calls segmented
by the biologist that represent the signal class we would like to detect
(2) a 2 second noise background file (noise_signal.wav) that represents the acoustic noise picked up by the hydrophone
(3) the continuous file (test_signal.wav) with unsegmented manatee calls mixed with background noise that lasts approximately 30 seconds. 

The sampling rate is 48 KHz

#### Steps

Overall goal: 
- Compare two approaches of time series segmentation

First steps:
- Take spectrograms of the data
    - Find where most of the power is in the signal
    

Approach 1:
- Train an NLMS filter using the MCC cost function for the manatee calls train data and the background noise
    - MCC likely better than MSE because of the high noise environment
- Look at frequency response of the model coefficients or the impulse response of the system to see if the system is doing what it is supposed to
    - High output for the correct frequency ranges (due to match filter)
- Input data into each model (match filters)
- Classify the signal based on which filter's output is higher
    - Make sure to normalize so the filters are outputting with the same magnitude
    - Normalize to max output of filter?

Approach 2:
- The time series is not memoryless (!! have to do something about this !!)
- Create model for manatee and background
- Then work with the prediction error
- SPRT test (assume the signal has two modes)
- Select thresholdsd for SPRT test to find the mode that the time series is on
- Apply CUSUM to find when this mode changes

Present results:
- Use an ROC to evaluate approaches (because this is a detection problem
    - Look at the difference between the two signals
    - Put threshold on the difference
    - Change threshold to go from detecting 
    
    
	
			
				□ One network per class model… create a match filter for manatee calls and one for background and the winner take all will choose the highest of the two and assign the correct label
				□ Instead of using this arrangement, you could also train a classifier with a state model that would discriminate between two states… we can do this too if you want but you should compare it with the arrangement he suggested
				□ His arrangement is not the best, but it fits better with the material that has been presented in the course
			§ You have to compare to approaches
				□ One approach is a linear filter
				□ Other approach
					® SPRT test, assume the signal has two modes
					® Come up with model for manatee and background
					® Select thresholds for SPRT test to find the mode that the time series is on
					® Apply the CUSUM algorithm to find when this mode changes
					® The time series is not memoryless (you have a signal with autocorrelation functions that are not delta functions)
					® So you have to come up with a model for the manatees and a model for the background and then do the SPRT test and the CUSUM test on the prediction error
						◊ This exactly fits one of the cases that we have discussed in our lecture
			§ MCC will probably perform better than MSE because the environment has a lot of noise… maximizing MCC prob better
				□ Could do both and see what's better
			§ Can do linear or nonlinear filters
			§ You should look at the data
				□ It is important that we do not apply algorithms to data that we don’t understand
			§ Important for you to take spectrograms of the data
				□ Find where most power is in the signal
				□ Look at frequency response of the model coefficients that you get or the impulse response of the system to see if the system is doing what it is supposed to do
				□ Important to check the quality of our models
				□ You are just building match filters, so you should expect things and look at the filters you are using
			§ Presentation of results
				□ This is a detection problem
				□ You should have a ROC
					® Best way to characterize the performance of your system because it is a detector
					® The threshold is the difference between the two signals, put threshold on difference, change threshold… goes from detecting all background and you should get to the point where all of the manatee calls you have in your test set should be detected
					® THERE ARE 16 MANATEE CALLS
						◊ Use your ear to determine where they are
						◊ You will probably find 15 of the 16 calls
						◊ How well does your detector agree with your ear

## Project Implementation

#### Import Libraries

In [1]:
import scipy.io
from scipy.fft import fft, fftfreq, rfft, rfftfreq
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
%load_ext autotime

time: 0 ns (started: 2021-04-10 19:54:52 -04:00)


#### Load files